### Imports

In [ ]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Move two directories up to reach the parent directory of 'Projects'
parent_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))

# Add the parent directory to the Python path
sys.path.append(parent_dir)

In [ ]:
from datasets.traffic_signs_preprocessed.datasets_preparing import *
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import cv2

### Previewing Original Dataset

In [ ]:
import os
print(os.getcwd())

In [ ]:
# Assume the current directory is 'Projects/P1'
# Move up two directories and then specify the path to the pickle file
with open("../../datasets/traffic_signs_preprocessed/data0.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
data.keys()

In [ ]:
X_train = data["x_train"]
X_valid = data["x_validation"]
X_test = data["x_test"]

y_train = data["y_train"]
y_valid = data["y_validation"]
y_test = data["y_test"]

In [ ]:
X_train.shape, X_valid.shape, X_test.shape

In [ ]:
total_size = len(X_train) + len(X_valid) + len(X_test)
train_prp = len(X_train) / total_size
valid_prp = len(X_valid) / total_size
test_prp = len(X_test) / total_size
print("Data Summary")
print("------------")
print(f"Train: {train_prp*100:.2f}%\nValid: {valid_prp*100:.2f}%\nTest: {test_prp*100:.2f}%")

In [ ]:
label_list = label_text("../../datasets/traffic_signs_preprocessed/label_names.csv")

In [ ]:
def plot_sample_image(X_train, y_train, label_list, index, alpha=1.5, beta=10):
    image = np.transpose(X_train[index], (1, 2, 0))

    # call convertScaleAbs function
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    sns.set_style("whitegrid")
    plt.imshow(adjusted)
    plt.title(f"Sign: {label_list[y_train[index]]}")
    plt.show()

In [ ]:
plot_sample_image(X_train, y_train, label_list, index=209, alpha=1.5, beta=10)

### Prepare Data

In [ ]:
# Train, Valid, Test Files
train_file = "../../datasets/traffic_signs_preprocessed/train.pickle"
valid_file = "../../datasets/traffic_signs_preprocessed/valid.pickle"
test_file = "../../datasets/traffic_signs_preprocessed/test.pickle"

In [ ]:
X_train, y_train, s_train, c_train = load_rgb_data(train_file)
X_valid, y_valid, s_valid, c_valid = load_rgb_data(valid_file)
X_test, y_test, s_test, c_test = load_rgb_data(test_file)

In [ ]:
total_size = len(X_train) + len(X_valid) + len(X_test)
train_prp = len(X_train) / total_size
valid_prp = len(X_valid) / total_size
test_prp = len(X_test) / total_size
print("Data Summary")
print("------------")
print(f"Train: {train_prp*100:.2f}%\nValid: {valid_prp*100:.2f}%\nTest: {test_prp*100:.2f}%")

### Load data and VGG Model

In [1]:
from data import *

In [3]:
train_dl, val_dl = get_data()

In [4]:
from model import *

In [5]:
model, loss_fn, optimizer = get_model()

/Users/sudharsangopalakrishnan/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sudharsangopalakrishnan/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
input_size = (3, 224, 224)
summary(model, input_size=input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [ ]:
train_losses, train_accuracies = [], []
val_accuracies = []
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_epoch_losses, train_epoch_accuracies = [], []
    val_epoch_accuracies = []

    for ix, batch in enumerate(iter(train_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss)
        print(ix)
    train_epoch_loss = np.array(train_epoch_losses).mean()

    for ix, batch in enumerate(iter(train_dl)):
        x, y = batch
        is_correct = accuracy(x, y, model)
        train_epoch_accuracies.extend(is_correct)
    train_epoch_accuracy = np.mean(train_epoch_accuracies)

    for ix, batch in enumerate(iter(val_dl)):
        x, y = batch
        val_is_correct = accuracy(x, y, model)
        val_epoch_accuracies.extend(val_is_correct)
    val_epoch_accuracy = np.mean(val_epoch_accuracies)

    train_losses.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_accuracies.append(val_epoch_accuracy)
    print(f"TL: {train_epoch_loss}, TA: {train_epoch_accuracy}, VA: {val_epoch_accuracy}")

In [ ]:
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline

In [ ]:
epochs = np.arange(5)+1
plt.plot(epochs, train_accuracies, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracies, 'r', label='Validation accuracy')
plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
plt.title('Training and validation accuracy with VGG16 \nand 1K training data points')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim(0.95,1)
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) \
                            for x in plt.gca().get_yticks()])
plt.legend()
plt.grid('off')
plt.show()